# Preparation of GWR - Data

## Libraries and Settings

In [1]:
# Libraries
import os
import json
import folium
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from datetime import date

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print(os.getcwd())

/Users/ricostaedeli/Documents/ZHAW/GIS/gis_unterricht/Gis_projekt


## Import latest GWR Data

In [8]:
def retrieveBuildingInformation(egid):
    url = "https://madd.bfs.admin.ch/eCH-0206"
    payload = f"""<?xml version="1.0" encoding="UTF-8"?>
    <eCH-0206:maddRequest xmlns:eCH-0058="http://www.ech.ch/xmlns/eCH-0058/5" xmlns:eCH-0206="http://www.ech.ch/xmlns/eCH-0206/2" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.ech.ch/xmlns/eCH-0206/2 eCH-0206-2-0-draft.xsd">
    <eCH-0206:requestHeader>
    <eCH-0206:messageId>myMessageId</eCH-0206:messageId>
    <eCH-0206:businessReferenceId>BFS/OFS/UST</eCH-0206:businessReferenceId>
    <eCH-0206:requestingApplication>
    <eCH-0058:manufacturer>soapUI</eCH-0058:manufacturer>
    <eCH-0058:product>Request-http</eCH-0058:product>
    <eCH-0058:productVersion>5.2.1</eCH-0058:productVersion>
    </eCH-0206:requestingApplication>
    <eCH-0206:requestDate>2021-02-22T16:15:47Z</eCH-0206:requestDate>
    </eCH-0206:requestHeader>
    <eCH-0206:requestContext>building</eCH-0206:requestContext>
    <eCH-0206:requestQuery>
        <eCH-0206:EGID>{egid}</eCH-0206:EGID>
    </eCH-0206:requestQuery>
    </eCH-0206:maddRequest>"""

    # headers
    headers = {
    'Content-Type': 'text/xml; charset=utf-8'
    }
    # POST request
    response = requests.request("POST", url, headers=headers, data=payload)

    #just for testing
    # temp = xml.dom.minidom.parseString(response.text) 
    # new_xml = temp.toprettyxml() 
    return response.text


# Create Dataframe for building entrances

In [2]:
import xml.etree.ElementTree as ET
import pandas as pd


def parse_xml_to_dataframe_entrances(xml_data):
    # Parse the XML data
    root = ET.fromstring(xml_data)
    
    # Extract namespaces from the XML data (assuming they are defined in the root element)
    namespaces = {node[0]: node[1] for node in root.tag[root.tag.find('{'):].split() if '}' in node}
    
    # If the dictionary extraction isn't correct, manually define the namespaces (based on your XML structure)
    namespaces = {
        '': 'http://www.ech.ch/xmlns/eCH-0206/2',  # default namespace (no prefix)
        'eCH-0058': 'http://www.ech.ch/xmlns/eCH-0058/5',
        'eCH-0129': 'http://www.ech.ch/xmlns/eCH-0129/5',
        'eCH-0007': 'http://www.ech.ch/xmlns/eCH-0007/6'
    }

    # Prepare a list to hold all parsed data
    all_data = []
    
    # Traverse through the XML structure
    for building in root.findall('.//{http://www.ech.ch/xmlns/eCH-0206/2}buildingItem'):
        EGID = building.find('{http://www.ech.ch/xmlns/eCH-0206/2}EGID').text if building.find('{http://www.ech.ch/xmlns/eCH-0206/2}EGID') is not None else None
        
        for entrance in building.findall('.//{http://www.ech.ch/xmlns/eCH-0206/2}buildingEntranceItem'):
            EGAID = entrance.find('.//{http://www.ech.ch/xmlns/eCH-0206/2}EGAID').text if entrance.find('.//{http://www.ech.ch/xmlns/eCH-0206/2}EGAID') is not None else None
            entranceNo = entrance.find('.//{http://www.ech.ch/xmlns/eCH-0206/2}buildingEntranceNo').text if entrance.find('.//{http://www.ech.ch/xmlns/eCH-0206/2}buildingEntranceNo') is not None else None
            isOfficialAddress = entrance.find('.//{http://www.ech.ch/xmlns/eCH-0206/2}isOfficialAddress').text if entrance.find('.//{http://www.ech.ch/xmlns/eCH-0206/2}isOfficialAddress') is not None else None
            east = entrance.find('.//{http://www.ech.ch/xmlns/eCH-0206/2}east').text if entrance.find('.//{http://www.ech.ch/xmlns/eCH-0206/2}east') is not None else None
            north = entrance.find('.//{http://www.ech.ch/xmlns/eCH-0206/2}north').text if entrance.find('.//{http://www.ech.ch/xmlns/eCH-0206/2}north') is not None else None
            
            for dwelling in entrance.findall('.//{http://www.ech.ch/xmlns/eCH-0206/2}dwellingItem'):
                EWID = dwelling.find('{http://www.ech.ch/xmlns/eCH-0206/2}EWID').text if dwelling.find('{http://www.ech.ch/xmlns/eCH-0206/2}EWID') is not None else None
                yearOfConstruction = dwelling.find('.//{http://www.ech.ch/xmlns/eCH-0206/2}yearOfConstruction').text if dwelling.find('.//{http://www.ech.ch/xmlns/eCH-0206/2}yearOfConstruction') is not None else None
                noOfRooms = dwelling.find('.//{http://www.ech.ch/xmlns/eCH-0206/2}noOfHabitableRooms').text if dwelling.find('.//{http://www.ech.ch/xmlns/eCH-0206/2}noOfHabitableRooms') is not None else None
                floor = dwelling.find('.//{http://www.ech.ch/xmlns/eCH-0206/2}floor').text if dwelling.find('.//{http://www.ech.ch/xmlns/eCH-0206/2}floor') is not None else None
                surfaceArea = dwelling.find('.//{http://www.ech.ch/xmlns/eCH-0206/2}surfaceAreaOfDwelling').text if dwelling.find('.//{http://www.ech.ch/xmlns/eCH-0206/2}surfaceAreaOfDwelling') is not None else None
                
                # Create a dictionary for the current dwelling
                dwelling_data = {
                    'EGID': EGID,
                    'EGAID': EGAID,
                    'EntranceNo': entranceNo,
                    'IsOfficialAddress': isOfficialAddress,
                    'CoordinateEast': east,
                    'CoordinateNorth': north,
                    'EWID': EWID,
                    'YearOfConstruction': yearOfConstruction,
                    'NumberOfRooms': noOfRooms,
                    'Floor': floor,
                    'SurfaceArea': surfaceArea
                }
                
                # Append the dwelling data dictionary to the all data list
                all_data.append(dwelling_data)
    
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(all_data)
    return df




# Build DataFrame for buildings

In [4]:
def parse_xml_with_namespaces(xml_data):
    # Parse XML data
    root = ET.fromstring(xml_data)
    
    # Define namespaces to use them with ElementTree find/findall
    namespaces = {
        'ns': 'http://www.ech.ch/xmlns/eCH-0206/2'
    }
    
    # Initialize a list to hold all building data
    buildings = []
    
    # Iterate through each building item in the XML
    for building_item in root.findall('.//ns:buildingItem', namespaces):
        building_dict = {}
        
        # Extract EGID and other building details
        building_dict['EGID'] = building_item.findtext('ns:EGID', namespaces=namespaces)
        building_details = building_item.find('ns:building', namespaces)
        
        if building_details is not None:
            building_dict['officialBuildingNo'] = building_details.findtext('ns:officialBuildingNo', namespaces=namespaces)
            coords = building_details.find('ns:coordinates', namespaces)
            if coords is not None:
                building_dict['east'] = coords.findtext('ns:east', namespaces=namespaces)
                building_dict['north'] = coords.findtext('ns:north', namespaces=namespaces)
                building_dict['originOfCoordinates'] = coords.findtext('ns:originOfCoordinates', namespaces=namespaces)
            
            building_dict['buildingStatus'] = building_details.findtext('ns:buildingStatus', namespaces=namespaces)
            building_dict['buildingCategory'] = building_details.findtext('ns:buildingCategory', namespaces=namespaces)
            building_dict['buildingClass'] = building_details.findtext('ns:buildingClass', namespaces=namespaces)
            
            date_of_construction = building_details.find('ns:dateOfConstruction', namespaces)
            if date_of_construction is not None:
                building_dict['dateOfConstruction'] = date_of_construction.findtext('ns:dateOfConstruction', namespaces=namespaces)
                building_dict['periodOfConstruction'] = date_of_construction.findtext('ns:periodOfConstruction', namespaces=namespaces)
            
            building_dict['surfaceAreaOfBuilding'] = building_details.findtext('ns:surfaceAreaOfBuilding', namespaces=namespaces)
            building_dict['numberOfFloors'] = building_details.findtext('ns:numberOfFloors', namespaces=namespaces)
            
            heating1 = building_details.find('ns:thermotechnicalDeviceForHeating1', namespaces)
            if heating1 is not None:
                building_dict['heatGeneratorHeating'] = heating1.findtext('ns:heatGeneratorHeating', namespaces=namespaces)
                building_dict['energySourceHeating'] = heating1.findtext('ns:energySourceHeating', namespaces=namespaces)
                building_dict['informationSourceHeating'] = heating1.findtext('ns:informationSourceHeating', namespaces=namespaces)
                building_dict['revisionDateHeating'] = heating1.findtext('ns:revisionDate', namespaces=namespaces)
            
            warmwater1 = building_details.find('ns:thermotechnicalDeviceForWarmWater1', namespaces)
            if warmwater1 is not None:
                building_dict['heatGeneratorHotWater'] = warmwater1.findtext('ns:heatGeneratorHotWater', namespaces=namespaces)
                building_dict['revisionDateWarmWater'] = warmwater1.findtext('ns:revisionDate', namespaces=namespaces)
            
            record_modification = building_details.find('ns:recordModification', namespaces)
            if record_modification is not None:
                building_dict['createDate'] = record_modification.findtext('ns:createDate', namespaces=namespaces)
                building_dict['updateDate'] = record_modification.findtext('ns:updateDate', namespaces=namespaces)
        
        # Add the dictionary to the list
        buildings.append(building_dict)
    
    # Create DataFrame
    df = pd.DataFrame(buildings)
    return df



In [31]:
egids = [28, 30, 190054641]
df_buildings = pd.DataFrame()
df_entrances = pd.DataFrame()

for egid in egids:
    xml_text = retrieveBuildingInformation(egid)
    temp_df_buildings = parse_xml_with_namespaces(xml_text)
    temp_df_entrances = parse_xml_to_dataframe_entrances(xml_text)
    
    # Update df_buildings by concatenating the new data
    df_buildings = pd.concat([df_buildings, temp_df_buildings], ignore_index=True)
    df_entrances = pd.concat([df_entrances, temp_df_entrances], ignore_index=True)



In [27]:
df_buildings

,EGID,officialBuildingNo,east,north,originOfCoordinates,buildingStatus,buildingCategory,buildingClass,dateOfConstruction,periodOfConstruction,surfaceAreaOfBuilding,numberOfFloors,heatGeneratorHeating,energySourceHeating,informationSourceHeating,revisionDateHeating,heatGeneratorHotWater,revisionDateWarmWater,createDate,updateDate
0,28,1383,2676600.358,1236018.001,901,1004,1020,1110,1944,8012,103,2,7410,7598,865,2020-03-18,7610,2020-03-18,2001-11-29,2021-03-15
1,30,1385,2676611.164,1236016.008,901,1004,1020,1110,1944,8012,86,2,7436,7540,860,2001-11-29,7650,2001-11-29,2001-11-29,2021-03-15
2,190054641,2058,2756887.247,1260263.585,901,1004,1020,1121,2004-12,8020,187,3,7430,7520,869,2005-01-04,7630,2005-01-04,2005-01-04,2022-10-13


In [26]:
df_entrances

,EGID,EGAID,EntranceNo,IsOfficialAddress,CoordinateEast,CoordinateNorth,EWID,YearOfConstruction,NumberOfRooms,Floor,SurfaceArea
0,28,100000361,3,1,2676601.51,1236016.196,1,1999,5,3100,64
1,30,100000363,5,1,2676615.418,1236016.226,1,1999,5,3100,50
2,190054641,101352297,7,1,2756886.251,1260267.57,1,2004,5,3101,125
3,190054641,101352297,7,1,2756886.251,1260267.57,2,2004,5,3102,121


In [25]:
df_merged = pd.merge(df_buildings, df_entrances, left_on='EGID', right_on='EGID', how='left')
df_merged.to_csv('relevant_buildings.csv')
df_merged

,EGID,officialBuildingNo,east,north,originOfCoordinates,buildingStatus,buildingCategory,buildingClass,dateOfConstruction,periodOfConstruction,...,EGAID,EntranceNo,IsOfficialAddress,CoordinateEast,CoordinateNorth,EWID,YearOfConstruction,NumberOfRooms,Floor,SurfaceArea
0,28,1383,2676600.358,1236018.001,901,1004,1020,1110,1944,8012,...,100000361,3,1,2676601.51,1236016.196,1,1999,5,3100,64
1,30,1385,2676611.164,1236016.008,901,1004,1020,1110,1944,8012,...,100000363,5,1,2676615.418,1236016.226,1,1999,5,3100,50
2,190054641,2058,2756887.247,1260263.585,901,1004,1020,1121,2004-12,8020,...,101352297,7,1,2756886.251,1260267.57,1,2004,5,3101,125
3,190054641,2058,2756887.247,1260263.585,901,1004,1020,1121,2004-12,8020,...,101352297,7,1,2756886.251,1260267.57,2,2004,5,3102,121


In [29]:
# Create geodataframe and calculate latitude and longitude
gdf = gpd.GeoDataFrame(df_merged, geometry=gpd.points_from_xy(df_merged['CoordinateEast'], df_merged['CoordinateNorth']), crs="EPSG:2056")

# Convert the Swiss LV95 coordinates to lat & lon
gdf = gdf.to_crs(epsg=4326)

# Get Latitude and Longitude
df_merged['latitude'] = gdf['geometry'].y
df_merged['longitude'] = gdf['geometry'].x

# Show data
df_merged

,EGID,officialBuildingNo,east,north,originOfCoordinates,buildingStatus,buildingCategory,buildingClass,dateOfConstruction,periodOfConstruction,...,IsOfficialAddress,CoordinateEast,CoordinateNorth,EWID,YearOfConstruction,NumberOfRooms,Floor,SurfaceArea,latitude,longitude
0,28,1383,2676600.358,1236018.001,901,1004,1020,1110,1944,8012,...,1,2676601.51,1236016.196,1,1999,5,3100,64,47.270604,8.450926
1,30,1385,2676611.164,1236016.008,901,1004,1020,1110,1944,8012,...,1,2676615.418,1236016.226,1,1999,5,3100,50,47.270603,8.451110
2,190054641,2058,2756887.247,1260263.585,901,1004,1020,1121,2004-12,8020,...,1,2756886.251,1260267.57,1,2004,5,3101,125,47.474453,9.520032
3,190054641,2058,2756887.247,1260263.585,901,1004,1020,1121,2004-12,8020,...,1,2756886.251,1260267.57,2,2004,5,3102,121,47.474453,9.520032


#

In [30]:
# Create the map
m = folium.Map(location=[df_merged['latitude'].mean(), df_merged['longitude'].mean()], zoom_start=15)

# Add points to the map
for idx, row in df_merged.iterrows():
    folium.Marker(location=([row['latitude'], 
                            row['longitude']]),
                  popup=row['buildingCategory']).add_to(m)

# Display the map
m